a = 1

In [ ]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython
from IPython.core.display import display

import librosa
import librosa.display
# import soundfile as sf

# our own stuff
import analog

# Introduction

Please check the first few [slides of the course](https://manuvazquez.github.io/assets/communications_theory/slides/analog_modulations.pdf) for this module, until *Types of Modulations*. They just review some ideas we saw during the [course introduction](https://manuvazquez.github.io/assets/communications_theory/slides/introduction.pdf) when talking about *Analog vs Digital communications systems*

# Amplitude Modulation (conventional AM)

An amplitude modulation is a kind of *linear* or *amplitude* (**analog**) modulation, i.e., the information signal is embedded in the amplitude of the signal (meaning the frequency and phase of the *carrier signal* stay constant). If we denote the information signal (also referred to as _modulat**ing**_ signal) by $x(t)$, then the _modulat**ed**_ signal is given by

$$
\large
y(t)
=
\left( 
    A_c +
    A_m
    x(t)
\right)
\cos (w_ct)
$$
where
* both $A_c$ and $A_m$ are (adjustable) modulation parameters 
* $w_c$ is the carrier frequency

The above signal can be expressed in a different way

$$
\large
y(t)
=
\left( 
    A_c +
    \frac{A_c}{A_c}
    A_m
    x(t)
\right)
\cos (w_ct)
=
A_c
\left(
    1 + mx(t)
\right)
\cos (w_ct)
$$
by defining the **modulation index**

$$
\large
m
=
\frac{A_m}{A_c}
$$
.

# Demodulation

If the signal is *normalized* (i.e., $|x(t)| \le 1$), looking at the above equation, demodulation is very easy if

$$
    \large
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$$

i.e., if the term multiplying the cosine is (at every time instant) non-negative. The reason is that whatever *positive* signal multiplies a rapidly varying cosine constitutes its so-called upper [envelope](https://en.wikipedia.org/wiki/Envelope_(waves)) (a smooth signal that outlines the extremes of a sinusoid), and simple/cheap/efficient hardware is available to extract the latter. Now, if the signal is at some point negative, then it cannot be recovered as the envelope of the signal. In our particular case, what do we need for the condition
$
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$
to hold? Above we have guaranteed that $|x(t)| \le 1$. Let us also assume that $A_c \ge 0$ (no need to go into details, but this is not a problem). Then,  we just need to choose $m$ so that $mx(t) \ge -1$, i.e., $ 0 < m \le 1$.

So, in summary, if the modulation index, $m$ is between $0$ and $1$, then the envelope of the modulated signal (easy to extract) is exactly
$
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$, and from the latter one can solve for $x(t)$ to recover the information signal. If $m>1$, then the envelope of the signal doesn't match anymore
$
    A_c
    \left(
        1 + mx(t)
    \right)
    \ge
    0
$
and the signal recovered with this envelope-based method is not correct. This is called **overmodulation**.

# An audio signal

We load the song *Reverie* by [\_ghost](http://ccmixter.org/files/_ghost/25389) (downloaded from [ccMixter](http://ccmixter.org/) under [Creative Commons licence](https://creativecommons.org/licenses/by/3.0/)).

In [ ]:
filename = pathlib.Path('_ghost_-_Reverie_(small_theme).mp3')
assert filename.exists()

In [ ]:
signal, sampling_rate = librosa.load(filename)
signal.shape

In [ ]:
print(f'Sampling rate is {sampling_rate}')

In [ ]:
normalized_signal, normalization_const = analog.normalize(signal, return_normalization_constant=True)

In [ ]:
# in order to force an unnormalized signal
amplified_signal = signal * 10

In [ ]:
# time axis
t = np.arange(len(signal)) / sampling_rate

In [ ]:
# sf_signal, sf_sample_rate = sf.read(filename.with_suffix('.wav'))
# print(sf_sample_rate, sf_signal.shape)

In [ ]:
%matplotlib widget

In [ ]:
# Parameters
w_c = 2 * np.pi * 1_000
A_m = 1.
A_c = 2.

In [ ]:
def make_plot_and_player(signal: np.ndarray) -> list:
    
    # modulation/demodulation
    am = analog.AmplitudeModulation(Am=A_m, Ac=A_c, carrier_freq=w_c)
    modulated_signal, *_ = am.modulate(t, signal)
    demodulated_signal = am.demodulate(modulated_signal)
    
    res = []

    # a `matplotlib` figure embedded in an `Output` widget
    figure_size = (6,8)
    output = widgets.Output()
    with output:
        fig, ax = plt.subplots(1, 1, figsize=figure_size)
    ax.plot(t, signal);
#     fig.set_label(' ')
    fig.canvas.header_visible = False
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    res.append(output)

    # an audio widget
    output = widgets.Output()
    with output:
        IPython.display.display(IPython.display.Audio(demodulated_signal, rate=sampling_rate))
    res.append(output)
    
    return res

**<font color='red'>Q1.</font>** Two versions of the audio signal are first modulated and demodulated using the above algebra. Look at the pictures. Can you guess which one is going to sound OK? If not sure, just play both of them for a few seconds. Which one sounds fine? Why not the other?

In [ ]:
widgets.HBox([widgets.VBox(make_plot_and_player(amplified_signal)), widgets.VBox(make_plot_and_player(signal))])

### Modulation

In [ ]:
# interval_of_interest = range(1_900,2_000)
# interval_of_interest = range(2_000,2_500)
# interval_of_interest = range(2_000,2_200)
interval_of_interest = range(100_000,100_200)

When you adjust one of the slides, please wait a few seconds (more or less depending on the *cloud* servers load) for the browser to refresh (it should flicker **twice**).

In [ ]:
common_properties = {'min': 0.1, 'max': 50.}
Am_slider_widget = widgets.FloatSlider(**common_properties, value=1., description='$A_m$')
Ac_slider_widget = widgets.FloatSlider(**common_properties, value=2, description='$A_c$')
ui = widgets.VBox([Am_slider_widget, Ac_slider_widget])

def f(Am: float, Ac: float):
    
    am = analog.AmplitudeModulation(Am=Am, Ac=Ac, carrier_freq=w_c)
    modulated_signal, envelope, cosine_factor = am.modulate(t, normalized_signal)
    demodulated_signal = am.demodulate(modulated_signal)
    
    # figure
    fig, ax = plt.subplots(1, 1, figsize=(15,8))
    ax.plot(
        t[interval_of_interest], normalized_signal[interval_of_interest], label='information (modulating) signal')
    ax.plot(
        t[interval_of_interest], demodulated_signal[interval_of_interest], label='demodulated signal',
        marker='P', markevery=5)
    ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left',fontsize='x-large')
    player = IPython.display.Audio(demodulated_signal, rate=sampling_rate)
    print(f'm = {am.m}')
    
    display(IPython.display.Audio(demodulated_signal, rate=sampling_rate))

out = widgets.interactive_output(f, {'Am': Am_slider_widget, 'Ac': Ac_slider_widget})
IPython.display.display(ui, out)

* Try different values of the parameters.
* Does overmodulation damage the signal? What is the required degree to actually be able to *hear the difference*?
* Provide examples of $A_m$ and $A_c$ for overmodulation
* Differences between the unnormalized signal and the normalized one (perception, modulation/demodulation)
* What happens with the unnormalized signal?
* When is noise perceived?